In [1]:
import pandas as pd

In [2]:
# Se  importan sólo las variables necesarias buscando optimizar la consulta.

steam_games = pd.read_parquet(r"C:\Users\roylo\OneDrive\Documentos\Data Science\Proyectos Individuales\Machine Learning Operations\Datasets\steam_games.parquet",columns=['id','price'])

steam_games.head()

,id,price
0,761140,4.99
1,643980,0.00
2,670290,0.00
3,767400,0.99
4,772540,3.99


In [3]:
# Se  importan sólo las variables necesarias buscando optimizar la consulta.

user_reviews = pd.read_parquet(r"C:\Users\roylo\OneDrive\Documentos\Data Science\Proyectos Individuales\Machine Learning Operations\Datasets\user_reviews.parquet",columns=['user_id','item_id','recommend'])

user_reviews.head()

,user_id,item_id,recommend
0,76561197970982479,1250,True
1,76561197970982479,22200,True
2,76561197970982479,43110,True
3,js41637,251610,True
4,js41637,227300,True


In [4]:
user_data = pd.merge(left=steam_games,right=user_reviews,left_on='id',right_on='item_id',how='inner')  # Se unen los datasets previamente obtenidos.

user_data.head()

,id,price,user_id,item_id,recommend
0,70,9.99,EizanAratoFujimaki,70,True
1,70,9.99,GamerFag,70,True
2,70,9.99,76561198020928326,70,True
3,70,9.99,Bluegills,70,True
4,70,9.99,76561198071955492,70,True


In [5]:
user_data = user_data[['user_id','item_id','price','recommend']]  # Se filtran sólo las variables útiles.

user_data.head()

,user_id,item_id,price,recommend
0,EizanAratoFujimaki,70,9.99,True
1,GamerFag,70,9.99,True
2,76561198020928326,70,9.99,True
3,Bluegills,70,9.99,True
4,76561198071955492,70,9.99,True


In [6]:
spent_money = user_data.groupby(['user_id'])['price'].sum()  # Se suman los precios de los artículos adquiridos por el usuario.

spent_money = spent_money.astype(str) # La suma del dinero se transforma a "str".

spent_money = spent_money + " USD" # A la suma de dinero se le agrega la divisa "USD".

total_items = user_data.groupby(['user_id'])['item_id'].count() # Se cuentan los items comprados por el usuario.

true_recommend = user_data[user_data['recommend']==True].groupby('user_id')['recommend'].count()  # Se cuentan los items que el usuario recomendó.

df1 = pd.merge(spent_money,total_items,on='user_id')

df2 = pd.merge(df1,true_recommend,on='user_id',how='left').fillna(0) # Se une el conteo de recomendación y se llenan los valores nulos, pues representan a los valores que no se recomendaron.

df2['% de recomendación'] = df2['recommend']/df2['item_id']*100   # Se obtiene el porcentaje de los items recomendados respecto al total de items adquiridos.

df2['% de recomendación'] = df2['% de recomendación'].astype(int).astype(str)

df2['% de recomendación'] = df2['% de recomendación'] + "%"

df2.head()

,price,item_id,recommend,% de recomendación
user_id,,,,
--000--,19.99 USD,1,1.0,100%
--ace--,4.99 USD,2,2.0,100%
--ionex--,9.99 USD,1,1.0,100%
-2SV-vuLB-Kg,64.97 USD,4,4.0,100%
-Beave-,19.99 USD,1,1.0,100%


In [7]:
user_data = df2[['price','% de recomendación','item_id']].rename(columns={'price':'Dinero Gastado','item_id':'Cantidad de Items'}) # Se renombran las variables.

user_data.head()

,Dinero Gastado,% de recomendación,Cantidad de Items
user_id,,,
--000--,19.99 USD,100%,1
--ace--,4.99 USD,100%,2
--ionex--,9.99 USD,100%,1
-2SV-vuLB-Kg,64.97 USD,100%,4
-Beave-,19.99 USD,100%,1


In [8]:
user_data.reset_index(inplace=True)

user_data.head()

In [10]:
user_data.to_parquet(r"C:\Users\roylo\OneDrive\Documentos\Data Science\Proyectos Individuales\Machine Learning Operations\Datasets\Apis\user_data\user_data.parquet",compression='snappy')